In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import matplotlib as mpl
import math
import pickle
import matplotlib.patches as mpatches
from tqdm import tqdm
from glob import glob

dat_path = os.getcwd()[:-4] + 'data/'
figures_path = os.getcwd()[:-4] + 'figures/'

text_font = 30

In [3]:
def axis_decor(ax, text_font, major_length, minor_length, linewidth):

    ax.spines['top'].set_linewidth(0)
    ax.spines['right'].set_linewidth(0)
    ax.spines['bottom'].set_linewidth(linewidth)
    ax.spines['left'].set_linewidth(linewidth)

    ax.tick_params(axis='both', which='major', labelsize=text_font, length=major_length, width=linewidth)
    ax.tick_params(axis='both', which='minor', labelsize=text_font, length=minor_length, width=linewidth)

#### 0. clean data with min-innings

In [4]:
def data_clean():

    # find all files with given prefix
    files = glob(dat_path + 'ODI_data_new_unmerged*.csv')

    odi_data = pd.read_csv(files[0])

    odi_data = odi_data.drop_duplicates()
    try:
        odi_data.drop(['Unnamed: 0'], axis=1, inplace=True)
        odi_data.drop(['Unnamed: 0.1'], axis=1, inplace=True)
    except:
        pass

    # Splitting the DataFrame into two: one for Team1 and another for Team2
    df_T1 = odi_data.filter(regex='_T1|Team1|search_ID|Match_ID|Match_Date|Venue|PoM|Winner', axis=1).copy()
    df_T2 = odi_data.filter(regex='_T2|Team2|search_ID|Match_ID|Match_Date|Venue|PoM|Winner', axis=1).copy()

    # Renaming columns to remove _T1 and _T2 suffixes
    df_T1.columns = df_T1.columns.str.replace('_T1', '')
    df_T2.columns = df_T2.columns.str.replace('_T2', '')

    df_T1.columns = df_T1.columns.str.replace('Team1', 'Team')
    df_T2.columns = df_T2.columns.str.replace('Team2', 'Team')

    # Adding columns to indicate batting order
    df_T1['Batting_Order'] = 'First'
    df_T2['Batting_Order'] = 'Second'

    # Concatenate the two DataFrames
    odi_final = pd.concat([df_T1, df_T2], axis=0).sort_values(by=['Match_ID', 'Batting_Order', 'bat_pos' ])

    # Resetting index
    odi_final.reset_index(drop=True, inplace=True)
    
    odi_final.to_csv(dat_path + f'ODI_data_cleaned_merged.csv', index=False)

    # return odi_final

# data_clean()

#### 1. renormalization of runs

In [5]:
def renormalization(norm_quant):

    odi_final = pd.read_csv(dat_path + f'ODI_data_cleaned_merged.csv')
    odi_final['Match_Date'] = odi_final['Match_Date'].str.replace(r'(\d)(st|nd|rd|th)', r'\1', regex=True)
    odi_final['Match_Date'] = pd.to_datetime(odi_final['Match_Date'], format='%d %B, %Y')


    global_mean = odi_final[norm_quant].mean()


    # Calculate the yearly mean for each year
    yearly_means = odi_final.groupby(odi_final['Match_Date'].dt.year)[norm_quant].mean()

    # Calculate the renormalization factor for each year
    renormalization_factors = global_mean / yearly_means

    # Apply the renormalization factor to each score
    odi_final['renormalized_bat_run'] = odi_final.apply(lambda x: x['bat_run'] * renormalization_factors.loc[x['Match_Date'].year], axis=1)

    odi_final['renormalized_runs'] = odi_final.apply(lambda x: x['Runs'] * renormalization_factors.loc[x['Match_Date'].year], axis=1)

    odi_final['renormalized_bowl_runs'] = odi_final.apply(lambda x: x['bowl_runs'] * renormalization_factors.loc[x['Match_Date'].year], axis=1)


    odi_final.to_csv(dat_path + f'ODI_data_cleaned_merged_renorm_{norm_quant}.csv', index=False)

# renormalization(norm_quant='Runs')
# renormalization(0, 'bat_run')
# renormalization(0, 'bowl_runs')

#### 2. team match number

In [6]:
def team_match_no(norm_quant):

    odi_final = pd.read_csv(dat_path + f'ODI_data_cleaned_merged_renorm_{norm_quant}.csv')

    odi_final['Match_Date'] = pd.to_datetime(odi_final['Match_Date'])
    odi_final = odi_final.sort_values(by='Match_ID')

    odi_final = odi_final.drop_duplicates()

    odi_final['team_match_num'] = np.nan

    odi_final = odi_final.sort_values(by=['Match_ID','Team'])

    team_list = np.unique(odi_final['Team'])
    match_no = {team: 0 for team in team_list}


    for i in tqdm(range(len(odi_final))):

        team = odi_final.iloc[i]['Team']

        if match_no[team] == 0:
            match_no[team] += 1
            odi_final.iloc[i, odi_final.columns.get_loc('team_match_num')] = match_no[team]
        else:
            if (odi_final.iloc[i]['Match_ID'] == odi_final.iloc[i-1]['Match_ID']) and (odi_final.iloc[i]['Team'] == odi_final.iloc[i-1]['Team']):
                odi_final.iloc[i, odi_final.columns.get_loc('team_match_num')] = match_no[team]
            else:
                match_no[team] += 1
                odi_final.iloc[i, odi_final.columns.get_loc('team_match_num')] = match_no[team]


    odi_final['team_match_num'] = odi_final['team_match_num'].astype(int)

    odi_final.to_csv(dat_path + f'ODI_data_cleaned_merged_renorm_{norm_quant}_with_team_match_num.csv', index=False)

# team_match_no(norm_quant='Runs')  

#### 3. player designations

bowlers: players who have bowled in at least 50% of their matches

batsmen: players who have batted at position 7 or above in at least 50% of their matches

all-rounders: players who are both bowlers and batsmen

In [7]:
def player_designation(norm_quant):

    odi_final = pd.read_csv(dat_path + f'ODI_data_cleaned_merged_renorm_{norm_quant}_with_team_match_num.csv')

    player_list = np.unique(odi_final['player_name'].astype(str))

    odi_final['batsmen'] = np.nan
    odi_final['bowler'] = np.nan
    odi_final['allrounder'] = np.nan

    for player in tqdm(player_list):
        player_ind = odi_final[odi_final['player_name'] == player].index
        temp_df = odi_final[odi_final['player_name'] == player]

        if len(temp_df) == 0:
            print(player)
            continue

        #count number of times bowl_overs is not a number
        bowl_overs_nan_count = temp_df['bowl_overs'].isna().sum()
        bowl_prop = 1-(bowl_overs_nan_count/len(temp_df))

        #count number of times bat_pos is less than or equal to 7
        bat_pos_count = len(temp_df[temp_df['bat_pos'] <= 7])
        bat_prop = bat_pos_count/len(temp_df)

        if bowl_prop >= 0.5:
            odi_final.loc[player_ind, 'bowler'] = 1
        if bat_prop >= 0.5:
            odi_final.loc[player_ind, 'batsmen'] = 1
        if (bowl_prop >= 0.5) and (bat_prop >= 0.5):
            odi_final.loc[player_ind, 'allrounder'] = 1


    # odi_final = odi_final.dropna(subset=['batsmen', 'bowler', 'allrounder'], how='all')


    odi_final.to_csv(dat_path + f'ODI_data_cleaned_merged_renorm_{norm_quant}_with_team_match_num_with_player_designation.csv', index=False)
        
# player_designation(norm_quant='Runs')

#### 4. opposition team wicket

In [8]:
def opp_team_wicket(norm_quant):

    odi_final = pd.read_csv(dat_path + f'ODI_data_cleaned_merged_renorm_{norm_quant}_with_team_match_num_with_player_designation.csv')

    odi_final['opp_wickets'] = np.nan

    match_id_list = np.unique(odi_final['Match_ID'])

    for match_id in tqdm(match_id_list):
        temp_df = odi_final[odi_final['Match_ID'] == match_id]
        team_list = np.unique(temp_df['Team'])

        if len(team_list) != 2:
            continue
        
        team1_ind = odi_final[(odi_final['Match_ID'] == match_id) & (odi_final['Team'] == team_list[0])].index
        team2_ind = odi_final[(odi_final['Match_ID'] == match_id) & (odi_final['Team'] == team_list[1])].index
        

        team1_df = temp_df[temp_df['Team'] == team_list[0]]
        team2_df = temp_df[temp_df['Team'] == team_list[1]]

        team1_wickets = team1_df['bowl_wickets'].sum()
        team2_wickets = team2_df['bowl_wickets'].sum()

        odi_final.loc[team1_ind, 'opp_wickets'] = team1_wickets
        odi_final.loc[team2_ind, 'opp_wickets'] = team2_wickets


    # odi_final = odi_final.dropna(subset=['opp_wickets'], how='all')

    odi_final['bat_pos'] = odi_final['bat_pos'].apply(lambda x: x - 11 if x > 11 else x)

    odi_final.to_csv(dat_path + f'ODI_data_cleaned_merged_renorm_{norm_quant}_with_team_match_num_with_player_designation.csv', index=False)


# opp_team_wicket(norm_quant='Runs')

100%|██████████| 4418/4418 [00:53<00:00, 81.90it/s]


#### 5. Number of matches with more than 50 overs

In [9]:
def more_than_50_overs(norm_quant):

    dat = pd.read_csv(dat_path + f'ODI_data_cleaned_merged_renorm_{norm_quant}_with_team_match_num_with_player_designation.csv')

    ### find number of matches before 1987
    dat['Match_Date'] = pd.to_datetime(dat['Match_Date'])
    dat = dat.sort_values(by='Match_ID')
    dat = dat.drop_duplicates(['Match_ID', 'Overs'])
    print(len(dat))
    # dat = dat[dat['Match_Date'].dt.year <= 1995]

    ### find number of matches where the number of overs is over 50 overs
    dat = dat[dat['Overs'] > 50]

    print(len(dat))

# more_than_50_overs(norm_quant='Runs')

##### 6. Number of players

In [10]:
def num_plrs(norm_quant):

    df = pd.read_csv(dat_path + f'ODI_data_cleaned_merged_renorm_{norm_quant}_with_team_match_num_with_player_designation.csv')

    print(len(df['player_name'].unique()))

    num_bowlers= np.count_nonzero(df.groupby('player_name')['bowler'].count().sort_values(ascending=False).values >=25)
    num_batsmen= np.count_nonzero(df.groupby('player_name')['batsmen'].count().sort_values(ascending=False).values >=25)
    num_allrounders = np.count_nonzero(df.groupby('player_name')['allrounder'].count().sort_values(ascending=False).values >=25)

    print('Number of bowlers:', num_bowlers)
    print('Number of batsmen:', num_batsmen)
    print('Number of allrounders:', num_allrounders)

num_plrs(norm_quant='Runs')

2863
Number of bowlers: 551
Number of batsmen: 580
Number of allrounders: 177


In [11]:
norm_quant = 'Runs'
df = pd.read_csv(dat_path + f'ODI_data_cleaned_merged_renorm_{norm_quant}_with_team_match_num_with_player_designation.csv')


print(len(np.unique(df['Match_ID'])))
print(len(np.unique(df['player_name'].astype(str))))

df['Match_Date'] = pd.to_datetime(df['Match_Date'])

min(df['Match_Date'])

4418
2863


Timestamp('1971-01-05 00:00:00')